In [1]:
# import pip
# sorted(["%s==%s" % (i.key, i.version) for i in pip.get_installed_distributions()])

In [2]:
# import os
# os.getcwd()

In [3]:
import tensorflow as tf
import numpy as np

# Import MINST data
from tensorflow.examples.tutorials.mnist import input_data

''' To classify images using a bidirectional reccurent neural network, we consider every image row as a sequence of pixels. Because MNIST image shape is 28*28px, we will then handle 28 sequences of 28 steps for every sample. '''

In [4]:
tf.reset_default_graph()

MNIST_dataset_path = 'MNIST_data/'
log_path = 'log'

# fake_data = False  # If true, uses fake data for unit testing
# dropout = 0.9  # Keep probability for training dropout

if tf.gfile.Exists(log_path):
    tf.gfile.DeleteRecursively(log_path)
tf.gfile.MakeDirs(log_path)

# mnist = input_data.read_data_sets(MNIST_dataset_path, one_hot = True, fake_data = fake_data)
mnist = input_data.read_data_sets(MNIST_dataset_path, one_hot = True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [5]:
# Parameters
learning_rate = 0.001
training_iters = 1000000
batch_size = 128
display_step = 10

# Network Parameters
n_input = 28 # MNIST data input (img shape: 28*28)
n_steps = 28 # timesteps
n_hidden = 128 # hidden layer num of features
n_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
x = tf.placeholder("float", [None, n_steps, n_input], name = 'InputData')
y = tf.placeholder("float", [None, n_classes], name = 'LabelData')

# Define weights
weights = {
    # Hidden layer weights => 2*n_hidden because of foward + backward cells
    'out': tf.Variable(tf.random_normal([2*n_hidden, n_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [6]:
def BiRNN(x, weights, biases):

    # Prepare data shape to match `bidirectional_rnn` function requirements
    # Current data input shape: (batch_size, n_steps, n_input)
    # Required shape: 'n_steps' tensors list of shape (batch_size, n_input)
    
    # Permuting batch_size and n_steps
    x = tf.transpose(x, [1, 0, 2])
    # Reshape to (n_steps*batch_size, n_input)
    x = tf.reshape(x, [-1, n_input])
    # Split to get a list of 'n_steps' tensors of shape (batch_size, n_input)
#     x = tf.split(x, n_steps, 0)
    x = tf.split(0, n_steps, x)
    
    # Define lstm cells with tensorflow
    # Forward direction cell
    lstm_fw_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden, forget_bias=1.0)
    # Backward direction cell
    lstm_bw_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden, forget_bias=1.0)

    # Get lstm cell output
    try:
        outputs, _, _ = tf.nn.bidirectional_rnn(lstm_fw_cell, lstm_bw_cell, x,
                                              dtype=tf.float32)
    except Exception: # Old TensorFlow version only returns outputs not states
        outputs = tf.nn.bidirectional_rnn(lstm_fw_cell, lstm_bw_cell, x,
                                        dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

In [7]:
with tf.name_scope('Model'):
    pred = BiRNN(x, weights, biases)

with tf.name_scope('Loss'):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))

with tf.name_scope('Optimizer'):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# # Evaluate model
with tf.name_scope('Accuracy'):
    correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# # Initializing the variables
init = tf.global_variables_initializer()

In [8]:
# Create a summary to monitor cost tensor
tf.summary.scalar("loss", cost)
# Create a summary to monitor accuracy tensor
tf.summary.scalar("accuracy", accuracy)
# Merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

In [9]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    
    # op to write logs to Tensorboard
    summary_writer = tf.summary.FileWriter(log_path, graph=tf.get_default_graph())
    
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Reshape data to get 28 seq of 28 elements
        batch_x = batch_x.reshape((batch_size, n_steps, n_input))
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
        if step % display_step == 0:
            
            # Run optimization op (backprop), cost op (to get loss value)
            # and summary nodes
            foo, bar, summary = sess.run([optimizer, cost, merged_summary_op],
                                     feed_dict={x: batch_x, y: batch_y})
            # Write logs at every iteration
            summary_writer.add_summary(summary, step)
            
            # Calculate batch accuracy
            acc = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y})
            # Calculate batch loss
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y})
            print ("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
        step += 1
    print ("Optimization Finished!")

    # Calculate accuracy for 128 mnist test images
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, n_steps, n_input))
    test_label = mnist.test.labels[:test_len]
    print ("Testing Accuracy:", \
        "{:.6f}".format(sess.run(accuracy, feed_dict={x: test_data, y: test_label})))
    
    print("Run the command line:\n" +
          "--> tensorboard --logdir=" + str(log_path) + "/" +
          "\nThen open http://0.0.0.0:6006/ into your web browser")

Iter 1280, Minibatch Loss= 1.544142, Training Accuracy= 0.46875
Iter 2560, Minibatch Loss= 1.269908, Training Accuracy= 0.61719
Iter 3840, Minibatch Loss= 0.951552, Training Accuracy= 0.67188
Iter 5120, Minibatch Loss= 0.716021, Training Accuracy= 0.79688
Iter 6400, Minibatch Loss= 0.486613, Training Accuracy= 0.82812
Iter 7680, Minibatch Loss= 0.852044, Training Accuracy= 0.69531
Iter 8960, Minibatch Loss= 0.520851, Training Accuracy= 0.84375
Iter 10240, Minibatch Loss= 0.431069, Training Accuracy= 0.82812
Iter 11520, Minibatch Loss= 0.276907, Training Accuracy= 0.93750
Iter 12800, Minibatch Loss= 0.461440, Training Accuracy= 0.83594
Iter 14080, Minibatch Loss= 0.295597, Training Accuracy= 0.93750
Iter 15360, Minibatch Loss= 0.230759, Training Accuracy= 0.94531
Iter 16640, Minibatch Loss= 0.355793, Training Accuracy= 0.92969
Iter 17920, Minibatch Loss= 0.209557, Training Accuracy= 0.93750
Iter 19200, Minibatch Loss= 0.173832, Training Accuracy= 0.93750
Iter 20480, Minibatch Loss= 0.16

https://www.tensorflow.org/versions/r0.10/api_docs/python/nn/#rnn

https://github.com/dennybritz/tf-rnn/blob/master/rnn_cell_wrappers.py.ipynb

https://www.tensorflow.org/how_tos/summaries_and_tensorboard/

https://www.tensorflow.org/how_tos/graph_viz/